# 检索机器人

### step1 加载预训练的向量模型

In [1]:
import torch

from typing import Optional
from datasets import load_dataset
from torch.nn import CosineEmbeddingLoss, CosineSimilarity
from transformers import AutoTokenizer, BertPreTrainedModel, BertModel

In [2]:
class BertForSimilarity(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.post_init()

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        A_input_ids, B_input_ids = input_ids[:, 0], input_ids[:, 1]
        A_attention_mask, B_attention_mask = attention_mask[:, 0], attention_mask[:, 1]
        A_token_type_ids, B_token_type_ids = token_type_ids[:, 0], token_type_ids[:, 1]

        A_outputs = self.bert(
            A_input_ids,
            attention_mask=A_attention_mask,
            token_type_ids=A_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        A_pooled_output = A_outputs[1]

        B_outputs = self.bert(
            B_input_ids,
            attention_mask=B_attention_mask,
            token_type_ids=B_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        B_pooled_output = B_outputs[1]

        similarity = CosineSimilarity()(A_pooled_output, B_pooled_output)

        loss = None
        if labels is not None:
            loss_func = CosineEmbeddingLoss(0.3)
            loss = loss_func(A_pooled_output, B_pooled_output, labels)
        output = (similarity, )
        return ((loss,) + output) if loss is not None else output
    

In [3]:
tokenizer = AutoTokenizer.from_pretrained("trained/models_for_seqAseqsimilarity/checkpoint-750")

In [4]:
model = BertForSimilarity.from_pretrained("trained/models_for_seqAseqsimilarity/checkpoint-750")

if torch.cuda.is_available():
    model = model.cuda()

### step2 加载数据

In [5]:
import pandas as pd
from tqdm import tqdm

dataset = pd.read_csv("../../datas/law_faq.csv")

In [6]:
dataset

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。
...,...,...
18208,在一个APP贷款，3500元，18期还款，算下来需要换6000元，请问贷款方是否违法？,18期就是18个月还6000那么，按1年12个月算的话，就是4000每个月，需要还款333....
18209,没交社保，只有工资，向哪个部门投诉,*司所在地的劳*局中的劳*监*大*。全称是人力资源与社会保障局，找到这个单位后找劳*监*办*...
18210,老赖欠钱不还，可以用坐牢抵债吗,肯定是要还的，判刑坐牢是针对他不还钱行为的惩罚，不是说坐牢抵债的，但如果一个人真的没钱你也没...
18211,我男朋友把我姐夫的车给偷了，我姐夫报案了，一开始我不知道他偷车了我知道后帮他逃跑我有罪吗,朋友。你男朋友把我姐夫的车给偷了，我姐夫报案了，一开始你不知道他偷车，我知道了后帮助你男朋友...


### step3 使用预训练模型将数据转化为向量

In [7]:
model.eval()
question_vectors = []

with torch.inference_mode():
    for idx in tqdm(range(0, len(dataset), 32)):
        sentence_list = dataset["title"][idx: idx + 32].to_list()
        inputs = tokenizer(
            sentence_list, 
            max_length=128, 
            truncation=True,
            padding=True,
            return_tensors="pt")
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        vector = model.bert(**inputs)[1]
        question_vectors.append(vector)

question_vectors = torch.concat(question_vectors, dim=0).cpu().numpy()

100%|██████████| 570/570 [01:17<00:00,  7.33it/s]


In [8]:
question_vectors.shape

(18213, 768)

### step4 使用向量数据库储存向量

In [10]:
import faiss

q_index = faiss.IndexFlatIP(768)
faiss.normalize_L2(question_vectors)
q_index.add(question_vectors)
q_index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x000001D4987938A0> >

### step5 检索向量

In [33]:
question = "寻衅滋事"

In [ ]:
with torch.inference_mode():
    inputs = tokenizer(question, max_length=128, truncation=True, padding=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    vector = model.bert(**inputs)[1]
    q_vector = vector.cpu().numpy()

In [16]:
faiss.normalize_L2(q_vector)
score, idx = q_index.search(q_vector, 10)
topk = dataset.values[idx[0].tolist()]

In [22]:
topk

array([['涉嫌寻衅滋事',
        '说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等

### step6 在检索的备选向量中再做匹配

因为向量数据库大范围检索是会有误差，为了增强结果的稳健性，对粗筛的结果再用Bert小范围检索一遍

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("models/macbert-base")
classify_model = AutoModelForSequenceClassification.from_pretrained("trained/models_for_seqcrossimilarity/checkpoint-250", num_labels=1)

In [25]:
if torch.cuda.is_available():
    classify_model = classify_model.cuda()

In [28]:
canidate = topk[:, 1]

In [44]:
qs = [question] * len(topk)
canidate = topk[:, 0].tolist()

with torch.inference_mode():
    inputs = tokenizer(qs, canidate, max_length=256, truncation=True, padding=True, return_tensors="pt")
    inputs = {k: v.to(classify_model.device) for k, v in inputs.items()}
    logits = classify_model(**inputs).logits.squeeze()
    result = logits.argmax(dim=-1).cpu().item()

In [45]:
qa = topk[result]
q, a = qa[0], qa[1]

In [47]:
q

'涉嫌寻衅滋事'

In [48]:
a

'说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等严重后果的;3、多次随意殴打他人的;4、持凶器随意殴